In [1]:
# | default_exp client

# Client

> base classes and functions for this library

In [2]:
# !pip install httpx

In [3]:
# | exporti
from __future__ import annotations

import os
from typing import Any, Union
from dataclasses import dataclass, field
from abc import abstractmethod, ABC

from urllib.parse import urlparse

import httpx
import json

from pprint import pprint
import domolibrary_extensions.utils as ut

In [4]:
# | hide
from nbdev.showdoc import show_doc

In [5]:
# | export


@dataclass
class Auth(ABC):
    """Base class for authentication"""

    @abstractmethod
    def generate_auth_header(self) -> dict:
        """Get the headers for the authentication"""
        pass

In [6]:
# | export
@dataclass
class ResponseGetData:
    """class for returning data from any route"""

    is_from_cache: bool
    is_success: bool

    status: int
    response: Any
    auth: Any = field(repr=False, default=None)

    def __post_init__(self):
        self.is_success = True if self.status >= 200 and self.status <= 399 else False

    @classmethod
    def _from_httpx(cls, res: httpx.Response, auth: Any = None):
        
        return cls(
            status=res.status_code,
            response= res.json(),
            is_success=res.is_success,
            is_from_cache=False,
            auth=auth,
        )
    
    @classmethod
    def _from_stream(cls, res: httpx.Response, content ,auth: Any = None,  ):
    
        if not res.is_success:
            content = res.json()
    
        return cls(
            status=res.status_code,
            response= content,
            is_success=res.is_success,
            is_from_cache=False,
            auth=auth,
        )

    @classmethod
    def _from_cache(cls, data: dict = None, auth: Any = None):
        return cls(
            status=200,
            response=data,
            is_success=True,
            is_from_cache=True,
            auth=auth,
        )

In [7]:
show_doc(ResponseGetData)

---

[source](https://github.com/jaewilson07/domolibrary_extensions/blob/main/domolibrary_extensions/client.py#L35){target="_blank" style="float:right; font-size:smaller"}

### ResponseGetData

>      ResponseGetData (is_from_cache:bool, is_success:bool, status:int,
>                       response:Any, auth:Any=None)

class for returning data from any route

In [8]:
ResponseGetData(
    status=200, response="hello world", is_from_cache=False, is_success=True
)

ResponseGetData(is_from_cache=False, is_success=True, status=200, response='hello world')

In [9]:
show_doc(ResponseGetData._from_httpx)

---

[source](https://github.com/jaewilson07/domolibrary_extensions/blob/main/domolibrary_extensions/client.py#L49){target="_blank" style="float:right; font-size:smaller"}

### ResponseGetData._from_httpx

>      ResponseGetData._from_httpx (res:httpx.Response, auth:Any=None)

# Cache Handlers

In [10]:
# | export
def get_cache(cache_path: str, debug_prn: bool = False) -> Union[dict, None]:
    """function for getting cached data from json file"""

    json_data = None
    ut.upsert_folder(folder_path=cache_path, debug_prn=debug_prn)

    try:
        with open(cache_path, "r", encoding="utf-8") as file:
            json_data = json.load(file)

    except (FileNotFoundError, json.JSONDecodeError) as e:
        with open(cache_path, "w+", encoding="utf-8") as file:
            pass
        json_data = None

    if json_data:
        if debug_prn:
            print(f"🚀 Using cached data in {cache_path}")

    return json_data


def update_cache(cache_path: str, data: Any, debug_prn : bool = False):
    
    ut.upsert_folder(cache_path)

    cache_path = ut.rename_filepath_to_match_datatype(data, cache_path)

    if debug_prn:
        print(f"updating {type(data)} content to {cache_path}")
        print(data)


    if isinstance(data, bytearray) or isinstance(data,bytes):
        with open(cache_path, "wb") as bf:
            return bf.write(data)

    with open(cache_path, "w+", encoding="utf-8") as fp:
        if isinstance(data, dict):
            return json.dump(data, fp)
        
        if isinstance(data, str):
            return fp.write(data)

In [11]:
show_doc(get_cache)

---

[source](https://github.com/jaewilson07/domolibrary_extensions/blob/main/domolibrary_extensions/client.py#L72){target="_blank" style="float:right; font-size:smaller"}

### get_cache

>      get_cache (cache_path:str, debug_prn:bool=False)

function for getting cached data from json file

In [12]:
test_cache_path = "../TEST/cache.json"

assert test_cache_path

get_cache(test_cache_path)

{'cases_time_series': [{'dailyconfirmed': '1',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '30 January 2020',
   'dateymd': '2020-01-30',
   'totalconfirmed': '1',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '0',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '31 January 2020',
   'dateymd': '2020-01-31',
   'totalconfirmed': '1',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '0',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '1 February 2020',
   'dateymd': '2020-02-01',
   'totalconfirmed': '1',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '1',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '2 February 2020',
   'dateymd': '2020-02-02',
   'totalconfirmed': '2',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '1',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '3 February 2020',
   'dateymd': '2020-

# Get Data

In [13]:
# | exporti
def prepare_fetch(
    url: str,
    params: dict = None,
    auth: Auth = None,
    headers: dict = None,
    body: dict = None,
):
    """base function to prepare a fetch operation"""

    headers = headers or {"Accept": "application/json"}

    if auth:
        headers = {**headers, **auth.generate_auth_header()}

    return headers, url, params, body

In [14]:
# | exporti
def _generate_cache_name(url):
    uparse = urlparse(url)

    return f"./CACHE/{''.join([uparse.netloc.replace('.', '_'), uparse.path.replace('.', '_')])}.json"

In [15]:
_generate_cache_name("https://app.asana.com/api/1.0/projects")

'./CACHE/app_asana_com/api/1_0/projects.json'

In [16]:
# | export
async def get_data(
    url: str,
    method: str,
    cache_path: str = None,
    is_ignore_cache: bool = False,
    headers: dict = None,
    params: dict = None,
    body=None,
    auth: Auth = None,
    parent_class: str = None,
    debug_api: bool = False,
    debug_prn: bool = False,
    client: httpx.AsyncClient = None,
    is_verify_ssl: bool = False,
) -> ResponseGetData:
    """wrapper for httpx Request library, always use with jiralibrary class"""

    cache_path = cache_path or _generate_cache_name(url)

    if not is_ignore_cache and cache_path:
        json_data = get_cache(cache_path=cache_path, debug_prn=debug_prn)

        if json_data:
            return ResponseGetData._from_cache(data=json_data, auth=auth)

    is_close_session = False if client else True
    client = client or httpx.AsyncClient(verify=is_verify_ssl)

    headers, url, params, body = prepare_fetch(
        url=url,
        params=params,
        auth=auth,
        headers=headers,
        body=body,
    )

    if debug_api:
        pprint(
            {
                "headers": headers,
                "url": url,
                "params": params,
                "body": body,
                "cache_file_path": cache_path,
                "debug_api": debug_api,
                "parent_class": parent_class,
            }
        )

    if method.upper() == "GET":
        res = await client.get(
            url=url,
            headers=headers,
            params=params,
            follow_redirects=True,
        )
    else:
        res = await getattr(client, method)(
            url=url,
            headers=headers,
            params=params,
            data=body,
        )

    if is_close_session:
        await client.aclose()

    rgd = ResponseGetData._from_httpx(res, auth=auth)

    if rgd.is_success:
        update_cache(cache_path=cache_path, data=rgd.response, debug_prn= debug_prn)

    return rgd

In [17]:
show_doc(get_data)

---

[source](https://github.com/jaewilson07/domolibrary_extensions/blob/main/domolibrary_extensions/client.py#L139){target="_blank" style="float:right; font-size:smaller"}

### get_data

>      get_data (url:str, method:str, cache_path:str=None,
>                is_ignore_cache:bool=False, headers:dict=None,
>                params:dict=None, body=None, auth:__main__.Auth=None,
>                parent_class:str=None, debug_api:bool=False,
>                debug_prn:bool=False, client:httpx.AsyncClient=None,
>                is_verify_ssl:bool=False)

wrapper for httpx Request library, always use with jiralibrary class

In [18]:
url = "https://api.covid19india.org/data.json"

try:
    print(await get_data(
        cache_path="../TEST/cache.json",
        url=url,
        method="GET",
        client=None,
        is_ignore_cache=True,
        debug_api=False,
        is_verify_ssl=False,
        debug_prn = True
    ))

except Exception as e:
    print(e)

In [22]:
#| export
        
async def get_data_stream(
    url: str,
    cache_path: str = None,
    is_ignore_cache: bool = False,
    headers: dict = None,
    params: dict = None,
    body=None,
    auth: Auth = None,
    parent_class: str = None,
    debug_api: bool = False,
    debug_prn: bool = False,
    client: httpx.AsyncClient = None,
    is_text: bool = False, # if false will interpret as bytes
    is_verify_ssl: bool = False,
) -> ResponseGetData:
    """wrapper for httpx Request library, always use with jiralibrary class"""

    cache_path = cache_path or _generate_cache_name(url)

    if not is_ignore_cache and cache_path:
        json_data = get_cache(cache_path=cache_path, debug_prn=debug_prn)

        if json_data:
            return ResponseGetData._from_cache(data=json_data, auth=auth)

    is_close_session = False if client else True

    client = client or httpx.AsyncClient(verify=is_verify_ssl)

    headers, url, params, body = prepare_fetch(
        url=url,
        params=params,
        auth=auth,
        headers=headers,
        body=body,
    )

    if debug_api:
        pprint(
            {
                "headers": headers,
                "url": url,
                "params": params,
                "body": body,
                "cache_file_path": cache_path,
                "debug_api": debug_api,
                "parent_class": parent_class,
            }
        )

    content = bytearray()

    async with client.stream(
        method="GET",
        url=url,
        headers=headers,
        params=params,
        follow_redirects=True,
    ) as res:
        if is_text:
            async for chunk in res.aiter_text():
                content += chunk
        
        else:
            async for chunk in res.aiter_bytes():
                content += chunk
            
    if is_close_session:
        await client.aclose()

    rgd = ResponseGetData._from_stream(res, auth=auth, content = content)

    if rgd.is_success:
        update_cache(cache_path=cache_path, data=rgd.response, debug_prn = debug_prn)

    return rgd

In [24]:
res = await get_data_stream(
    url = "https://api.covid19india.org/data.json",
    debug_prn = True,
    cache_path= '../TEST/cache_bytes.txt',
    is_text = False,
    debug_api = False
)

{'upsert_folder': '/root/GitHub/gdoc_sync/nbs/TEST', 'is_exist': True}
<class 'bytearray'>
updating <class 'bytearray'> content to ../TEST/cache_bytes
bytearray(b'{\n\t"cases_time_series": [\n\t\t{\n\t\t\t"dailyconfirmed": "1",\n\t\t\t"dailydeceased": "0",\n\t\t\t"dailyrecovered": "0",\n\t\t\t"date": "30 January 2020",\n\t\t\t"dateymd": "2020-01-30",\n\t\t\t"totalconfirmed": "1",\n\t\t\t"totaldeceased": "0",\n\t\t\t"totalrecovered": "0"\n\t\t},\n\t\t{\n\t\t\t"dailyconfirmed": "0",\n\t\t\t"dailydeceased": "0",\n\t\t\t"dailyrecovered": "0",\n\t\t\t"date": "31 January 2020",\n\t\t\t"dateymd": "2020-01-31",\n\t\t\t"totalconfirmed": "1",\n\t\t\t"totaldeceased": "0",\n\t\t\t"totalrecovered": "0"\n\t\t},\n\t\t{\n\t\t\t"dailyconfirmed": "0",\n\t\t\t"dailydeceased": "0",\n\t\t\t"dailyrecovered": "0",\n\t\t\t"date": "1 February 2020",\n\t\t\t"dateymd": "2020-02-01",\n\t\t\t"totalconfirmed": "1",\n\t\t\t"totaldeceased": "0",\n\t\t\t"totalrecovered": "0"\n\t\t},\n\t\t{\n\t\t\t"dailyconfirmed": "1"

In [ ]:
# | export
looper_offset_params = {"offset": "offset", "limit": "limit"}


async def looper(
    url,
    client: httpx.AsyncClient,
    auth: Auth,
    arr_fn,
    params: dict = None,
    offset=0,
    limit=50,
    offset_params: dict = None,
    debug_loop: bool = False,
    debug_api: bool = False,
    debug_prn: bool = False,
    method="GET",
    is_verify_ssl: bool = False,
    is_ignore_cache: bool = False,
    cache_path: str = None,
    **kwargs
):
    offset_params = offset_params or looper_offset_params

    cache_path = cache_path or _generate_cache_name(url)

    if not is_ignore_cache and cache_path:
        json_data = get_cache(cache_path=cache_path, debug_prn=debug_prn)

        if json_data:
            return ResponseGetData._from_cache(data=json_data, auth=auth)

    final_array = []
    keep_looping = True

    while keep_looping:
        new_params = params.copy() if params else {}

        new_params = {
            **new_params,
            offset_params["offset"]: offset,
            offset_params["limit"]: limit,
        }

        if debug_loop:
            print(new_params)

        res = await get_data(
            is_ignore_cache=True,
            auth=auth,
            url=url,
            method=method,
            params=new_params,
            debug_api=debug_api,
            debug_prn=debug_prn,
            client=client,
            is_verify_ssl=is_verify_ssl,
            **kwargs
        )

        new_array = arr_fn(res)

        if not new_array or len(new_array) == 0:
            keep_looping = False

        final_array += new_array
        offset += limit

    res.response = final_array

    if res.is_success:
        update_cache(cache_path=cache_path, data=res.response, debug_prn=debug_prn)

    return res

In [ ]:
#| export

class BaseError_Validation(Exception):
    def __init__(self, message):
        super().__init__(message)

class BaseError(Exception):
    def __init__(self, 
                 instance = None,
                 entity_id = None, 
                 message = None,
                 res : ResponseGetData= None
                 ):
        

        if not (instance and message) and not res:
            raise BaseError_Validation('must include instance and message or ResponseGetData class')
        
        message = message or res.response['Message']

        if entity_id:
            message = f"{message} for {entity_id}"

        if res:
            message = f"status {res.status} || {message} in {instance or res.auth.instance}"
        super().__init__(
            message
        )

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export('./client.ipynb')